In [24]:
%matplotlib inline
import numpy as np
import h5py 
from sklearn.metrics import f1_score,jaccard_similarity_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as pRf1
import glob
import cv2
from scipy.optimize import linear_sum_assignment
import sys
from skimage import io
############### Visualization

from ipywidgets import Button, Layout
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display,clear_output,Video
from copy import copy
import numpy.ma as ma
import matplotlib.colors as colors
################# Load modules
sys.path.insert(0,'/media/DATA/jbonato/astro_segm/AstroSS/modules/')

from score_fun import *
N = 256
M=256

# Dataset-1 vs. consensus

In [2]:
### uery files
GT_path = '/media/DATA/jbonato/astro_segm/set1/dataset/'
res_path = '/media/DATA/jbonato/astro_segm/Results/D1/'

masks_GT = glob.glob(GT_path+'*.*')
masks_GT.sort()
masks_GT

['/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_001_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_002_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_003_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_004_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_005_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_006_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_007_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_008_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_009_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_010_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_011_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_012_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_013_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_014_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_015_nf.hdf5',
 '/media/D

In [25]:
GT_mat = np.zeros((256,256,2))
Res_mat = np.zeros_like(GT_mat)

h_res = np.empty((24,3))
soma_res = []
proc_res=[]
fov=0
for GT_small in masks_GT:#[j:j+1]
    name = GT_small[-17:-8]+'D1.hdf5'
    # loading GT
    dset= h5py.File(GT_small,'r') 
    GT_mat[:,:,0] =  np.asarray(dset['Values'])
    GT_mat[:,:,1] =  np.asarray(dset['Values_soma'])
    GT_mat[:,:,0]-=GT_mat[:,:,1]
    GT_mat[GT_mat<0]=0
    GT_mat[GT_mat>1]=1


    # loading Res
    dset= h5py.File(res_path+name,'r') 
    Res_mat[:,:,0] =  np.asarray(dset['Values'])
    Res_mat[:,:,1] =  np.asarray(dset['Values_soma'])
    

    aa = score_cw(GT_mat,Res_mat)
    
    proc_res.append(aa[:,:3,:])
    soma_res.append(np.mean(aa[:,:3,:],axis=0))

    a,b,c = true_pos_l(GT_mat[:,:,1],Res_mat[:,:,1],threshold=0.5,N=256)
    h_res[fov,0]=a
    h_res[fov,1]=b
    h_res[fov,2]=c

    fov+=1

detection = h_res.copy()
f1_net = np.zeros_like(detection)
f1_net[:,0] = detection[:,0]/detection[:,2]
f1_net[:,1] = detection[:,0]/detection[:,1]
f1_net[:,2] = 2*(f1_net[:,0]*f1_net[:,1])/(f1_net[:,0]+f1_net[:,1]) 
mean_f1_net = np.mean(f1_net,axis=0)
std_f1_net = np.std(f1_net,axis=0)

final_proc = proc_res[0]
for r in range(1,len(proc_res)):
    final_proc = np.vstack((final_proc,proc_res[r]))
mean_val = np.mean(final_proc,axis=0)
print('Somata: \n P: {:.2f}, R:  {:.2f}, F1:  {:.2f}'.format(mean_val[0,0],mean_val[1,0],mean_val[2,0]))
print('Processes \n P: {:.2f}, R:  {:.2f}, F1:  {:.2f}'.format(mean_val[0,1],mean_val[1,1],mean_val[2,1]))
print(mean_f1_net)

NameError: name 'masks_GT' is not defined

# Results Visualization

In [26]:
def draw_cont(mask):
    N,M,c = mask.shape
    sm_c = np.zeros((N,M))
    pr_c = np.zeros((N,M))
    contours, hierarchy = cv2.findContours(np.uint8(mask[:,:,1]), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for j in contours:
        sm_c[j[:,0,1],j[:,0,0]]=1
        
    contours, hierarchy = cv2.findContours(np.uint8(mask[:,:,0]), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for j in contours:
        pr_c[j[:,0,1],j[:,0,0]]=1

    return np.where(sm_c==1),np.where(pr_c==1)#sm_c,pr_c#

def draw_on_Im(image,c_sm,c_pr):
    image_pl = image.copy()
    image_pl = image_pl.astype(np.float64)
    maxim = image_pl.max()
    image_pl/=maxim
    image_pl[c_sm]=4
    image_pl[c_pr]=10
    return image_pl


def get_im(mask,image):
    sm_net,pr_net = draw_cont(mask)
    image_net = draw_on_Im(image,sm_net,pr_net)
    return image_net


In [33]:
num  = widgets.IntSlider(
    value=7,
    min=1,
    max=25,
    step=1,
    description='FOV',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


mapped = widgets.Dropdown(
    options=[('inferno',plt.cm.inferno), ('viridis',plt.cm.viridis), ('Greys',plt.cm.gray)],
    value=plt.cm.gray,
    description='Cmap:',
    disabled=False,
)
projection =  widgets.Dropdown(
    options=['median', 'max', 'mean'],
    value='median',
    description='Projection:',
    disabled=False,
)


button = widgets.Button(
    description='Plot',
)
out=widgets.Output(layout=Layout(width='500px', height='500px'))
out2=widgets.Output(layout=Layout(width='500px', height='500px'))
button=widgets.Button(description='Next')
vbox=widgets.VBox(children=(out,num,mapped,projection,button))
hbox=widgets.HBox(children=(vbox,out2))
display(hbox)

def plot2(b=None):
    if num.value==20:
        with out:
            display('error no image present')
    else:
        path = '/media/DATA/jbonato/astro_segm/set1/'+str(num.value)+'/'
        if len(str(num.value))==1:
            zer='00'
        else:
            zer='0'
        res_path = '/media/DATA/jbonato/astro_segm/Results/D1/'
        name = 'SMALL_'+zer+str(num.value)+'D1.hdf5'
        Res_mat = np.zeros((N,M,2))
        dset= h5py.File(res_path+name,'r') 
        Res_mat[:,:,0] =  np.asarray(dset['Values'])
        Res_mat[:,:,1] =  np.asarray(dset['Values_soma'])

        it = glob.glob(path+'*')
        stack = io.imread(it[0])
        video = np.empty((stack.shape[0],256,256),dtype=np.uint8)
        for j in range(stack.shape[0]):
            img = stack[j,:,:]
            bl = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            video[j,:,:]=bl
        
        imageio.mimwrite('test2.mp4',video, fps=30);
        with out2:
            clear_output(wait=True)
            display(Video('test2.mp4',width=500,height=500))
        if projection.value=='median':
            im = np.median(stack,axis=0)
        elif projection.value == 'mean':
            im = np.mean(stack,axis=0)
        else:
            im = np.amax(stack,axis=0)
        imC = get_im(Res_mat,im)
        f, ax = plt.subplots(1, 1, figsize=(8, 8))
        c = ['snow','violet']
        palette = copy(mapped.value)
        palette.set_over(c[0],1.0)
        palette.set_bad(c[1],1.0)
        zm = ma.masked_where(imC>5,imC)
        ax.imshow(zm,cmap=palette,norm=colors.Normalize(vmin=0,vmax=1.0))
        ax.axis('off')
        with out:
            clear_output(wait=True)
            display(f)
    
button.on_click(plot2)

